In [1]:
import os, time, re, json, subprocess
import pandas as pd
import importlib.util as il

if None in [il.find_spec('python-ulid'), il.find_spec('pyperclip')]:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'python-ulid']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pyperclip']);
    
from ulid import ULID
import pyperclip

def gen_ulid():
    return str(ULID.from_timestamp(time.time()))

def pprint(dict):
    print(json.dumps(dict, indent=2))

def convert_coord(c):
    c = str(c)
    j = len(c) - 6
    d = int(c[0:2 + j])
    m = int(c[2 + j:4 + j])
    s = float(c[4 + j:6 + j] + '.' + c[6 + j:])
    q = 1 if j == 0 else -1
    coord = round(q * (d + m / 60 + s / 3600), 6)
    
    return coord

NameError: name 'subprocess' is not defined

In [ ]:
downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")

csv_filenames = [
    "RWY_PAIR_CONFIG.csv",
    "RPC_RUNWAY.csv",
    "RPC_RUNWAY_2.csv",
    "RPC_RUNWAY_3.csv"
]

dataframes = {}

for filename in csv_filenames:
    file_path = os.path.join(downloads_folder, filename)
    try:
        df = pd.read_csv(file_path)
        dataframes[filename] = df
        print(f"Loaded '{filename}' with {len(df)} rows.")
    except FileNotFoundError:
        print(f"File '{filename}' not found in Downloads folder.")
    except Exception as e:
        print(f"Error reading '{filename}': {e}")

rpc0 = dataframes.get("RWY_PAIR_CONFIG.csv")
rwy1 = dataframes.get("RPC_RUNWAY.csv")
rwy2 = dataframes.get("RPC_RUNWAY_2.csv")
rwy3 = dataframes.get("RPC_RUNWAY_3.csv")

In [ ]:
RPC_SCRATCH = {'FLE': ['FL', 'FVL', 'FAL', 'FR', 'FVR', 'FAR'], 
               'FLW': ['FL', 'FVL', 'FAL', 'FR', 'FVR', 'FAR']}

rpcs = []

counter = 1
for index, row in rpc0.iterrows():
    a = {}
    rid = row['RPC ID']
    
    a['id'] = gen_ulid()
    a['index'] = counter
    a['airportId'] = row['Airport ID']
    if row['Stagger Mode'] == '+':
        a['positionSymbolTie'] = '/'
        a['positionSymbolStagger'] = '+'
    else:
        a['positionSymbolTie'] = '+'
        a['positionSymbolStagger'] = '/'

    for i in range(0, 2):
        r_type = 'M'
        if i == 1:
            r_type = 'S'
        
        r = {}
        d1 = rwy1[(rwy1['Airport ID'] == row['Airport ID']) & 
            (rwy1['RPC ID'] == rid) & (rwy1['rwy_type'] == r_type)].iloc[0]
        rpc_num = d1['#.']
        d2 = rwy2[rwy2['#.'] == rpc_num].iloc[0]
        d3 = rwy3[rwy3['#.'] == rpc_num].iloc[0]
        # print(d1, d2, d3, sep='\n\n')
        
        r['runwayId'] = d1['Runway ID']
        r['headingTolerance'] = int(d1['Heading Tol'])
        r['nearSideHalfWidth'] = float(d3['NS HW'])
        r['farSideHalfWidth'] = float(d3['FS HW'])
        r['nearSideDistance'] = float(d3['NS Dist'])
        if r['nearSideDistance'] == 0:
            r['nearSideDistance'] = 0.01
        r['regionLength'] = float(d3['Region Len'])
        r['targetReferencePoint'] = {'lat': convert_coord(d1['Tgt Lat']), 'lon': -1 * convert_coord(d1['Tgt Long'])}
        r['targetReferenceLineHeading'] = float(d2['Tgt Angle'])
        r['targetReferenceLineLength'] = int(d2['Tgt Length'])
        r['targetReferencePointAltitude'] = int(d2['Tgt Alt'])
        r['imageReferencePoint'] = {'lat': convert_coord(d1['Img Lat']), 'lon': -1 * convert_coord(d2['Img Long'])}
        r['imageReferenceLineHeading'] = float(d2['Img Angle'])
        r['imageReferenceLineLength'] = int(d2['Img Length'])
        r['tieModeOffset'] = float(d2['Tie Offset'])
        if r['tieModeOffset'] == 0:
            r['tieModeOffset'] = 0.01
        r['descentPointDistance'] = int(d2['Descent Dist'])
        if r['descentPointDistance'] == 0:
            r['descentPointDistance'] = 0.01
        r['descentPointAltitude'] = int(d3['Descent Alt'])
        r['abovePathTolerance'] = int(d3['Above Tol'])
        if r['abovePathTolerance'] > 99:
            r['abovePathTolerance'] = 99
        r['belowPathTolerance'] = int(d3['Below Tol'])
        if r['belowPathTolerance'] > 99:
            r['belowPathTolerance'] = 99
        r['defaultLeaderDirection'] = str(d1['Orientation'])
        r['scratchpadPatterns'] = []
        if a['airportId'] in RPC_SCRATCH:
            r['scratchpadPatterns'] = RPC_SCRATCH[a['airportId']]

        if i == 0:
            a['masterRunway'] = r
        else:
            a['slaveRunway'] = r

    rpcs.append(a)

    if counter == 1:
        pprint(a)
    
    counter += 1

pyperclip.copy(json.dumps(rpcs, indent=2))